# **8. Ensemble** **de** **los** **modelos** **de** **detección** **de** **ingredientes**

A la vista de los resultados obtenidos en las distintas fases iterativas del reentrenaiento del modelo de detección de inredientes, se opta por ensamblar los dos modelos que mejores resultados dan:

- '*best_new_2.pt*': tiene un desempeño notable en el reconocimiento de las clases 'potato' y 'onion', pero más pobre para la clase 'egg', especialmente en aquellas imágenes que muestran huevos crudos.

- '*best_new_5.pt*': funciona bastante mejor en la clase 'egg', pero tiene un desempeño inferior en las demás clases.


In [ ]:
pip install ultralytics torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 26.2 MB/s eta 0:00:00


In [ ]:
from google.colab import files

# Esto abrirá un diálogo para elegir tu archivo .pt desde tu PC
uploaded = files.upload()

Saving best_new_2.pt to best_new_2.pt


Para ensamblar los modelos 'best_new_2' y 'best_new_5' se utiliza un algoritmo basado en el promedio ponderado de pesos. Para ello se utilizan las siguientes funciones:
- ***check_compat***: verifica que dos modelos cargados sean compatibles para combinarse, comprobando que tengan el mismo número de clases, nombres de clase y arquitectura.
- ***average_state_dicts***: genera un nuevo diccionario de pesos promediando, de forma ponderada, los parámetros de dos modelos siempre que coincidan en claves y formas.
- ***main***: combina dos modelos cargados promediando sus pesos, valida la compatibilidad y guarda el nuevo modelo resultante

In [ ]:
import torch
from ultralytics import YOLO

# ==== CONFIG ====
MODEL1_PATH = "best_new_2.pt"
MODEL2_PATH = "best_new_5.pt"
OUT_PATH    = "new_model.pt"

# Promedio ponderado: alpha y beta (alpha=1, beta=1 -> promedio simple)
ALPHA = 1.0
BETA  = 1.0
# =================

def check_compat(m1: YOLO, m2: YOLO):
    # Mismo número de clases
    nc1 = getattr(m1.model, 'nc', None) or len(getattr(m1.model, 'names', []))
    nc2 = getattr(m2.model, 'nc', None) or len(getattr(m2.model, 'names', []))
    assert nc1 == nc2, f"nc distintos: {nc1} vs {nc2}"

    # Mismos nombres de clase
    names1 = m1.model.names
    names2 = m2.model.names
    assert names1 == names2, "Los nombres/clases no coinciden exactamente."

    # Misma arquitectura (yaml)
    y1 = getattr(m1.model, 'yaml', None)
    y2 = getattr(m2.model, 'yaml', None)
    # y puede ser dict, comparamos claves esenciales
    if isinstance(y1, dict) and isinstance(y2, dict):
        for k in ['anchors','nc','depth_multiple','width_multiple','backbone','head']:
            if k in y1 or k in y2:
                assert y1.get(k, None) == y2.get(k, None), f"Config '{k}' difiere."
    else:
        # si no hay yaml detallado, al menos comparamos shapes de state_dict
        pass

def average_state_dicts(sd1, sd2, alpha=1.0, beta=1.0):
    assert sd1.keys() == sd2.keys(), "Los state_dict tienen claves distintas."
    out = {}
    total = alpha + beta
    for k in sd1.keys():
        t1, t2 = sd1[k], sd2[k]
        assert t1.shape == t2.shape, f"Shape distinto en '{k}': {t1.shape} vs {t2.shape}"
        if t1.dtype.is_floating_point:
            out[k] = (t1.to(torch.float32) * alpha + t2.to(torch.float32) * beta) / total
        else:
            # para enteros/booleanos (p.ej. num_batches_tracked) nos quedamos con el del primero
            out[k] = t1
    return out

def main():
    m1 = YOLO(MODEL1_PATH)
    m2 = YOLO(MODEL2_PATH)
    check_compat(m1, m2)

    sd1 = m1.model.state_dict()
    sd2 = m2.model.state_dict()

    avg_sd = average_state_dicts(sd1, sd2, ALPHA, BETA)

    # Cargamos el promedio en una copia del primero (misma arquitectura)
    m_new = YOLO(MODEL1_PATH)  # reutilizamos estructura
    missing, unexpected = m_new.model.load_state_dict(avg_sd, strict=False)
    assert not missing and not unexpected, f"Missing: {missing}, Unexpected: {unexpected}"

    # Guardar en formato Ultralytics
    # Opción A (recomendada): usar API save de Ultralytics
    m_new.save(OUT_PATH)

    print(f"✅ Guardado: {OUT_PATH}")
    print("Sugerencia: valida el modelo resultante para asegurar que el promedio mantiene el rendimiento.")

if __name__ == "__main__":
    main()


✅ Guardado: new_model.pt
Sugerencia: valida el modelo resultante para asegurar que el promedio mantiene el rendimiento.


In [ ]:
from google.colab import files
files.download("/content/new_model.pt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>